# Qiskit ExactReciprocal

This notebook demonstrates the Qiskit `ExactReciprocal` operation.

* [Qiskit ExactReciprocal documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ExactReciprocal)

The documentation states that the `ExactReciprocal` operation takes the state $|x\rangle|0\rangle$ to:
$$
\cos(\frac{1}{x})|x\rangle|0\rangle + \sin(\frac{1}{x})|x\rangle|1\rangle
$$

As well as the number of qubits for the input value, the operation takes two further parameters:

* `scaling`: Scaling factor $s$ of the reciprocal function, i.e. to compute $s/x$.
* `neg_vals` (Boolean): Whether $x$ might represent negative values. In this case the first qubit is the sign, with $|1\rangle$ for negative and $|0\rangle$ for positive. For the negative case it is assumed that the remaining string represents $1-x$. This is because $e^{-2\pi ix}=e^{2\pi i(1-x)}$ for $x\in[0,1]$.

Jack Morgan's HHL circuit uses `neg_vals=True`.

How does the `ExactReciprocal` operation work? Let's examine some examples.

In [68]:
import math
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import ExactReciprocal, StatePreparation
from qiskit.quantum_info import Statevector

def run_statevector(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    return simulator.run(aer_circuit).result().get_statevector()

x = np.array([1, 1])
x = x / np.linalg.norm(x)
num_state_qubits = int(x.size / 2)
state_preparation = StatePreparation(Statevector(x))

x_reg = QuantumRegister(num_state_qubits, 'x')
a_reg = QuantumRegister(1, 'a')
circuit = QuantumCircuit(a_reg, x_reg)
circuit.append(state_preparation, x_reg)

# circuit.draw(output='mpl')
# statevector = run_statevector(circuit)
# statevector.draw(output='latex')

exact_reciprocal = ExactReciprocal(num_state_qubits=num_state_qubits, scaling=1/2)
circuit.append(exact_reciprocal, x_reg[::-1] + [a_reg[0]])
# circuit.draw(output='mpl')

statevector = run_statevector(circuit)
statevector.draw(output='latex')

<IPython.core.display.Latex object>

# ExactReciprocal for input $|0\rangle$

|Scale| Statevector | $\theta=\frac{1}{x}$ |
|---|---|---|
| 1 | $\|00\rangle$ | 0 |
| 2 | $\|00\rangle$ | 0 |
| 4 | $\|00\rangle$ | 0 |
| 8 | $\|00\rangle$ | 0 |

# ExactReciprocal for input $|1\rangle$

|Scale| Statevector result | $\theta=\frac{1}{x}$ | 
|--:|---|---|
| 1 | $\|10\rangle$ | $0,\pi$ |
| 2 | $\|11\rangle$ | $\frac{\pi}{2}$ |
| 4 | $\frac{\sqrt{3}}{2}\|10\rangle + \frac{1}{2}\|11\rangle$ | $\frac{\pi}{6}$ |
| 8 | $\frac{\sqrt{15}}{4}\|10\rangle + \frac{1}{4}\|11\rangle$ | 0.2527 |
| 16 | $\frac{\sqrt{63}}{8}\|10\rangle + \frac{1}{8}\|11\rangle$ | 0.1253 |

# ExactReciprocal for input $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$

|Scale| Statevector | $\theta=\frac{1}{x}$ |
|---|---|---|
| 1 | $\frac{1}{\sqrt{2}}(\|00\rangle+\|10\rangle)$ | 0 |
| 2 | $\frac{1}{\sqrt{2}}(\|00\rangle+\|11\rangle)$ | ? |
| 4 | $\frac{1}{\sqrt{2}}\|00\rangle+\frac{\sqrt{6}}{4}\|10\rangle+\frac{\sqrt{2}}{4}\|11\rangle$ | ? |
| 8 | $\frac{1}{\sqrt{2}}\|00\rangle+\frac{\sqrt{30}}{8}\|10\rangle+\frac{1}{4\sqrt{2}}\|11\rangle$ | ? |